# 13. Biblioteca de Dispositivos

Este notebook demonstra como usar a biblioteca de dispositivos pré-definidos do Pulsim.

## Conteúdo
1. Visão geral da biblioteca
2. Diodos comerciais
3. MOSFETs comerciais
4. IGBTs comerciais
5. Switches ideais e reais
6. Comparação de dispositivos
7. Exemplos práticos

In [ ]:
import pulsim
from pulsim import devices
import numpy as np
import matplotlib.pyplot as plt

print(f"Pulsim version: {pulsim.__version__}")
print(f"\nDispositivos disponíveis no módulo 'devices':")
print([x for x in dir(devices) if not x.startswith('_')])

## 1. Visão Geral da Biblioteca

A biblioteca `pulsim.devices` contém parâmetros de dispositivos comerciais reais, permitindo simulações precisas sem precisar consultar datasheets.

### Categorias de Dispositivos

| Categoria | Função | Descrição |
|-----------|--------|------------|
| **Diodos** | `diode_*()` | Retificadores, Schottky, fast recovery, SiC |
| **MOSFETs** | `mosfet_*()` | N-channel, P-channel, GaN |
| **IGBTs** | `igbt_*()` | Propósito geral, alta velocidade, alta tensão |
| **Switches** | `switch_*()` | Ideal, relé, SSR |

## 2. Biblioteca de Diodos

### Diodos Disponíveis

In [ ]:
# Lista de diodos disponíveis
diodes = {
    '1N4007': devices.diode_1N4007(),
    '1N4148': devices.diode_1N4148(),
    '1N5819': devices.diode_1N5819(),
    'MUR860': devices.diode_MUR860(),
    'C3D10065A': devices.diode_C3D10065A(),
}

print("Biblioteca de Diodos:")
print("="*80)
print(f"{'Modelo':<12} {'Tipo':<15} {'Is (A)':<12} {'n':<6} {'Rs (Ω)':<10} {'Vbr (V)':<8} {'tt (s)'}")
print("-"*80)

diode_types = {
    '1N4007': 'Retificador',
    '1N4148': 'Sinal rápido',
    '1N5819': 'Schottky',
    'MUR860': 'Fast Recovery',
    'C3D10065A': 'SiC Schottky',
}

for name, d in diodes.items():
    print(f"{name:<12} {diode_types[name]:<15} {d.is_:<12.2e} {d.n:<6.2f} {d.rs:<10.4f} {d.bv if hasattr(d, 'bv') else 'N/A':<8} {d.tt if hasattr(d, 'tt') else 'N/A'}")

In [ ]:
# Comparar curvas I-V dos diodos
def diode_iv_curve(params, v_range):
    """Calcula corrente do diodo usando equação de Shockley."""
    vt = 0.026  # Thermal voltage at 300K
    i = params.is_ * (np.exp(v_range / (params.n * vt)) - 1)
    return i

v = np.linspace(0, 1.0, 1000)

plt.figure(figsize=(12, 5))

# Subplot 1: Escala linear
plt.subplot(1, 2, 1)
for name, d in diodes.items():
    i = diode_iv_curve(d, v)
    plt.plot(v, i * 1000, label=name, linewidth=2)

plt.xlabel('Tensão (V)')
plt.ylabel('Corrente (mA)')
plt.title('Curvas I-V dos Diodos (Escala Linear)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 100)

# Subplot 2: Escala log
plt.subplot(1, 2, 2)
for name, d in diodes.items():
    i = diode_iv_curve(d, v)
    plt.semilogy(v, i, label=name, linewidth=2)

plt.xlabel('Tensão (V)')
plt.ylabel('Corrente (A)')
plt.title('Curvas I-V dos Diodos (Escala Log)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(1e-12, 10)

plt.tight_layout()
plt.show()

print("\nObservações:")
print("- Schottky (1N5819, C3D10065A): Menor queda de tensão direta")
print("- Retificador (1N4007): Maior queda, mas alta tensão reversa")
print("- Fast Recovery (MUR860): Compromisso entre velocidade e queda")

## 3. Biblioteca de MOSFETs

### MOSFETs Disponíveis

In [ ]:
# Lista de MOSFETs disponíveis
mosfets = {
    'IRF540N': devices.mosfet_IRF540N(),
    'IRFZ44N': devices.mosfet_IRFZ44N(),
    'IRF9540': devices.mosfet_IRF9540(),
    'BSC0902NS': devices.mosfet_BSC0902NS(),
    'EPC2001C': devices.mosfet_EPC2001C(),
}

print("Biblioteca de MOSFETs:")
print("="*90)
print(f"{'Modelo':<12} {'Tipo':<8} {'Vth (V)':<10} {'Rds_on (mΩ)':<14} {'Cgs (pF)':<12} {'Cgd (pF)':<12} {'Body Diode'}")
print("-"*90)

for name, m in mosfets.items():
    tipo = 'NMOS' if m.type == pulsim.MOSFETType.NMOS else 'PMOS'
    body = 'Sim' if m.body_diode else 'Não'
    print(f"{name:<12} {tipo:<8} {m.vth:<10.1f} {m.rds_on*1000:<14.2f} {m.cgs*1e12:<12.0f} {m.cgd*1e12:<12.0f} {body}")

In [ ]:
# Comparação de Rds_on e capacitâncias
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Nomes e valores
names = list(mosfets.keys())
rds_on = [m.rds_on * 1000 for m in mosfets.values()]  # mOhm
cgs = [m.cgs * 1e12 for m in mosfets.values()]  # pF
cgd = [m.cgd * 1e12 for m in mosfets.values()]  # pF

colors = ['#2ecc71', '#3498db', '#9b59b6', '#e74c3c', '#f39c12']

# Rds_on
axes[0].bar(names, rds_on, color=colors)
axes[0].set_ylabel('Rds_on (mΩ)')
axes[0].set_title('Resistência On-State')
axes[0].tick_params(axis='x', rotation=45)
for i, v in enumerate(rds_on):
    axes[0].text(i, v + 2, f'{v:.1f}', ha='center', fontsize=9)

# Cgs
axes[1].bar(names, cgs, color=colors)
axes[1].set_ylabel('Cgs (pF)')
axes[1].set_title('Capacitância Gate-Source')
axes[1].tick_params(axis='x', rotation=45)

# Cgd (Miller cap)
axes[2].bar(names, cgd, color=colors)
axes[2].set_ylabel('Cgd (pF)')
axes[2].set_title('Capacitância Gate-Drain (Miller)')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\nObservações:")
print("- BSC0902NS: Ultra-baixo Rds_on (2.1mΩ) - ideal para DC-DC de alta eficiência")
print("- EPC2001C (GaN): Capacitâncias muito baixas - switching muito rápido")
print("- IRF9540 (PMOS): Rds_on mais alto que NMOS equivalente")

## 4. Biblioteca de IGBTs

### IGBTs Disponíveis

In [ ]:
# Lista de IGBTs disponíveis
igbts = {
    'IRG4PC40UD': devices.igbt_IRG4PC40UD(),
    'IRG4BC30KD': devices.igbt_IRG4BC30KD(),
    'IKW40N120H3': devices.igbt_IKW40N120H3(),
}

print("Biblioteca de IGBTs:")
print("="*95)
print(f"{'Modelo':<14} {'Vth (V)':<10} {'Vce_sat (V)':<12} {'Rce_on (Ω)':<12} {'tf (ns)':<10} {'tr (ns)':<10} {'Cies (pF)'}")
print("-"*95)

for name, g in igbts.items():
    print(f"{name:<14} {g.vth:<10.1f} {g.vce_sat:<12.2f} {g.rce_on:<12.3f} {g.tf*1e9:<10.0f} {g.tr*1e9:<10.0f} {g.cies*1e12:<.0f}")

print("\nAplicações típicas:")
print("- IRG4PC40UD: Motor drives, UPS (propósito geral)")
print("- IRG4BC30KD: Conversores ressonantes (alta velocidade)")
print("- IKW40N120H3: Inversores solares, industrial (1200V)")

## 5. Biblioteca de Switches

### Switches Disponíveis

In [ ]:
# Lista de switches disponíveis
switches = {
    'Ideal': devices.switch_ideal(),
    'Relay': devices.switch_relay(),
    'SSR': devices.switch_ssr(),
}

print("Biblioteca de Switches:")
print("="*60)
print(f"{'Modelo':<10} {'Ron (Ω)':<15} {'Roff (Ω)':<15} {'Vth (V)'}")
print("-"*60)

for name, s in switches.items():
    print(f"{name:<10} {s.ron:<15.2e} {s.roff:<15.2e} {s.vth:.1f}")

print("\nCaracterísticas:")
print("- Ideal: Para simulações conceituais (Ron = 1µΩ)")
print("- Relay: Modelo de relé mecânico (Ron = 100mΩ)")
print("- SSR: Solid-state relay (Ron = 20mΩ)")

## 6. Exemplo Prático: Retificador com Diferentes Diodos

In [ ]:
def simulate_rectifier(diode_params, name):
    """Simula retificador de meia onda com diodo especificado."""
    circuit = pulsim.Circuit()
    
    # Fonte AC 12V RMS, 60Hz
    sine = pulsim.SineWaveform()
    sine.amplitude = 12 * np.sqrt(2)  # Peak
    sine.frequency = 60
    circuit.add_voltage_source_waveform("Vac", "ac", "gnd", sine)
    
    # Diodo
    circuit.add_diode("D1", "ac", "dc", diode_params)
    
    # Carga
    circuit.add_resistor("Rload", "dc", "gnd", 100)
    
    # Simular
    opts = pulsim.SimulationOptions()
    opts.tstop = 0.05  # 3 ciclos
    opts.dt = 10e-6
    
    return pulsim.simulate(circuit, opts)

# Simular com diferentes diodos
test_diodes = {
    '1N4007 (Retificador)': devices.diode_1N4007(),
    '1N5819 (Schottky)': devices.diode_1N5819(),
    'C3D10065A (SiC)': devices.diode_C3D10065A(),
}

results = {}
for name, params in test_diodes.items():
    results[name] = simulate_rectifier(params, name)

# Plotar
plt.figure(figsize=(14, 5))

# Formas de onda
plt.subplot(1, 2, 1)
for name, result in results.items():
    data = result.to_dict()
    t = np.array(data['time']) * 1000
    plt.plot(t, data['signals']['V(dc)'], label=name, linewidth=1.5)

plt.xlabel('Tempo (ms)')
plt.ylabel('Tensão DC (V)')
plt.title('Tensão de Saída do Retificador')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 50)

# Zoom no pico para ver queda de tensão
plt.subplot(1, 2, 2)
for name, result in results.items():
    data = result.to_dict()
    t = np.array(data['time']) * 1000
    v_dc = np.array(data['signals']['V(dc)'])
    # Pegar região do pico
    mask = (t > 3.5) & (t < 5.5)
    plt.plot(t[mask], v_dc[mask], label=name, linewidth=2)

plt.axhline(y=12*np.sqrt(2), color='k', linestyle='--', alpha=0.5, label='Vpeak (ideal)')
plt.xlabel('Tempo (ms)')
plt.ylabel('Tensão DC (V)')
plt.title('Zoom no Pico - Queda de Tensão no Diodo')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Análise
print("Análise de Queda de Tensão:")
print("="*50)
v_peak_ideal = 12 * np.sqrt(2)
for name, result in results.items():
    data = result.to_dict()
    v_dc = np.array(data['signals']['V(dc)'])
    v_peak = np.max(v_dc)
    v_drop = v_peak_ideal - v_peak
    print(f"{name}: Vpeak = {v_peak:.2f}V, Queda = {v_drop:.3f}V")

## 7. Exemplo Prático: Buck Converter com Diferentes MOSFETs

In [ ]:
def simulate_buck(mosfet_params, diode_params, name):
    """Simula conversor Buck com MOSFET e diodo especificados."""
    circuit = pulsim.Circuit()
    
    # Parâmetros
    Vin = 24
    fsw = 100e3
    duty = 0.5
    
    # Fonte de entrada
    circuit.add_voltage_source("Vin", "vin", "gnd", Vin)
    
    # PWM para gate
    pwm = pulsim.PWMWaveform()
    pwm.v_off = 0
    pwm.v_on = 10
    pwm.frequency = fsw
    pwm.duty = duty
    circuit.add_voltage_source_waveform("Vpwm", "gate", "gnd", pwm)
    
    # MOSFET como switch controlado por tensão
    # Usamos switch para simplificar (MOSFET real requer driver)
    sw = pulsim.SwitchParams()
    sw.ron = mosfet_params.rds_on
    sw.roff = 1e9
    sw.vth = mosfet_params.vth
    circuit.add_switch("Q1", "vin", "sw", "gate", "gnd", sw)
    
    # Diodo de freewheeling
    circuit.add_diode("D1", "gnd", "sw", diode_params)
    
    # Filtro LC
    circuit.add_inductor("L1", "sw", "out", 100e-6)
    circuit.add_capacitor("Cout", "out", "gnd", 100e-6)
    
    # Carga
    circuit.add_resistor("Rload", "out", "gnd", 10)
    
    # Simular
    opts = pulsim.SimulationOptions()
    opts.tstop = 0.001
    opts.dt = 20e-9
    
    sim = pulsim.Simulator(circuit, opts)
    result = sim.run_transient()
    losses = sim.power_losses()
    
    return result, losses

# Testar diferentes combinações
test_configs = {
    'IRF540N + 1N4007': (devices.mosfet_IRF540N(), devices.diode_1N4007()),
    'IRFZ44N + 1N5819': (devices.mosfet_IRFZ44N(), devices.diode_1N5819()),
    'BSC0902NS + C3D10065A': (devices.mosfet_BSC0902NS(), devices.diode_C3D10065A()),
}

results = {}
losses = {}
for name, (mosfet, diode) in test_configs.items():
    results[name], losses[name] = simulate_buck(mosfet, diode, name)

# Plotar resultados
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Tensão de saída
for name, result in results.items():
    data = result.to_dict()
    t = np.array(data['time']) * 1e6
    axes[0].plot(t, data['signals']['V(out)'], label=name, linewidth=1.5)

axes[0].set_ylabel('V(out) [V]')
axes[0].set_title('Buck Converter - Comparação de Dispositivos')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Ripple (zoom)
for name, result in results.items():
    data = result.to_dict()
    t = np.array(data['time']) * 1e6
    v = np.array(data['signals']['V(out)'])
    mask = t > 900  # Últimos 100us
    axes[1].plot(t[mask], v[mask], label=name, linewidth=1.5)

axes[1].set_xlabel('Tempo (µs)')
axes[1].set_ylabel('V(out) [V]')
axes[1].set_title('Zoom no Ripple')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Análise de perdas
print("Análise de Perdas e Eficiência:")
print("="*70)

# Potência de saída esperada
Vout_expected = 24 * 0.5  # 12V
Pout_expected = Vout_expected**2 / 10  # 14.4W

for name, loss in losses.items():
    data = results[name].to_dict()
    vout = np.array(data['signals']['V(out)'])
    vout_avg = np.mean(vout[-500:])  # Últimos pontos
    pout = vout_avg**2 / 10
    
    total_loss = loss.total_loss()
    cond_loss = loss.conduction_loss
    sw_loss = loss.switching_loss()
    
    eff = pout / (pout + total_loss) * 100 if (pout + total_loss) > 0 else 0
    
    print(f"\n{name}:")
    print(f"  Vout médio: {vout_avg:.2f} V")
    print(f"  Pout: {pout:.2f} W")
    print(f"  Perdas condução: {cond_loss*1000:.2f} mW")
    print(f"  Perdas switching: {sw_loss*1000:.2f} mW")
    print(f"  Perdas totais: {total_loss*1000:.2f} mW")
    print(f"  Eficiência: {eff:.1f}%")

## 8. Criando Parâmetros Customizados

Você pode modificar parâmetros da biblioteca ou criar novos dispositivos.

In [ ]:
# Baseado no IRF540N, criar versão com menor Rds_on (simulando dispositivo mais novo)
custom_mosfet = devices.mosfet_IRF540N()  # Começar com parâmetros do IRF540N

print("IRF540N Original:")
print(f"  Rds_on: {custom_mosfet.rds_on*1000:.1f} mΩ")
print(f"  Vth: {custom_mosfet.vth} V")
print(f"  Cgs: {custom_mosfet.cgs*1e12:.0f} pF")

# Modificar para versão melhorada
custom_mosfet.rds_on = 0.020  # 20mΩ ao invés de 44mΩ
custom_mosfet.cgs = 2000e-12  # Menor capacitância
custom_mosfet.cgd = 200e-12

print("\nMOSFET Customizado:")
print(f"  Rds_on: {custom_mosfet.rds_on*1000:.1f} mΩ")
print(f"  Cgs: {custom_mosfet.cgs*1e12:.0f} pF")
print(f"  Cgd: {custom_mosfet.cgd*1e12:.0f} pF")

In [ ]:
# Criar diodo ideal para comparação
ideal_diode = pulsim.DiodeParams()
ideal_diode.ideal = True  # Usar modelo ideal

# Criar diodo customizado baseado no datasheet
custom_diode = pulsim.DiodeParams()
custom_diode.is_ = 5e-9      # Corrente de saturação
custom_diode.n = 1.8          # Fator de idealidade
custom_diode.rs = 0.03        # Resistência série
custom_diode.vt = 0.026       # Tensão térmica
custom_diode.ideal = False

print("Diodo Customizado:")
print(f"  Is: {custom_diode.is_:.2e} A")
print(f"  n: {custom_diode.n}")
print(f"  Rs: {custom_diode.rs*1000:.0f} mΩ")

## 9. Tabela Resumo de Aplicações

| Aplicação | Diodo Recomendado | MOSFET Recomendado | IGBT Recomendado |
|-----------|-------------------|--------------------|-----------------|
| Retificador 60Hz | 1N4007 | - | - |
| SMPS (100kHz+) | 1N5819 / MUR860 | IRF540N / IRFZ44N | - |
| DC-DC alta eficiência | C3D10065A | BSC0902NS | - |
| Motor Drive (< 10kHz) | MUR860 | - | IRG4PC40UD |
| Inversor Solar | C3D10065A | - | IKW40N120H3 |
| GaN Applications | - | EPC2001C | - |
| Conversor Ressonante | MUR860 | EPC2001C | IRG4BC30KD |

## Resumo

Neste notebook aprendemos:

1. **Biblioteca de Diodos**: 1N4007, 1N4148, 1N5819, MUR860, C3D10065A
2. **Biblioteca de MOSFETs**: IRF540N, IRFZ44N, IRF9540, BSC0902NS, EPC2001C
3. **Biblioteca de IGBTs**: IRG4PC40UD, IRG4BC30KD, IKW40N120H3
4. **Biblioteca de Switches**: Ideal, Relay, SSR
5. **Como usar**: `from pulsim import devices` e `devices.diode_1N4007()`
6. **Customização**: Modificar parâmetros ou criar novos dispositivos

A biblioteca facilita simulações realistas sem precisar consultar datasheets!